In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [12]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [13]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=False,
    c1=1e-4,
    tau=0.5,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.30757418274879456
epoch:  1, loss: 0.03823048993945122
epoch:  2, loss: 0.02052762545645237
epoch:  3, loss: 0.013146094977855682
epoch:  4, loss: 0.00920481700450182
epoch:  5, loss: 0.008544151671230793
epoch:  6, loss: 0.008072946220636368
epoch:  7, loss: 0.007920420728623867
epoch:  8, loss: 0.007875592447817326
epoch:  9, loss: 0.007734591141343117
epoch:  10, loss: 0.007684823125600815
epoch:  11, loss: 0.007662080228328705
epoch:  12, loss: 0.0076121985912323
epoch:  13, loss: 0.007584067527204752
epoch:  14, loss: 0.007540901657193899
epoch:  15, loss: 0.007516184821724892
epoch:  16, loss: 0.007510055787861347
epoch:  17, loss: 0.0074190436862409115
epoch:  18, loss: 0.007374202366918325
epoch:  19, loss: 0.00725740659981966
epoch:  20, loss: 0.0071539198979735374
epoch:  21, loss: 0.007144456263631582
epoch:  22, loss: 0.0063970647752285
epoch:  23, loss: 0.00634212838485837
epoch:  24, loss: 0.005457266233861446
epoch:  25, loss: 0.005418885964900255
epoc

In [14]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.8833358338063181
Test metrics:  R2 = 0.8498516952569561


In [15]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=True,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].cpu().detach().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.035894494503736496
epoch:  1, loss: 0.033560965210199356
epoch:  2, loss: 0.031222915276885033
epoch:  3, loss: 0.028881406411528587
epoch:  4, loss: 0.026570569723844528
epoch:  5, loss: 0.011423084884881973
epoch:  6, loss: 0.009013116359710693
epoch:  7, loss: 0.008268359117209911
epoch:  8, loss: 0.0071680280379951
epoch:  9, loss: 0.0062040844932198524
epoch:  10, loss: 0.005235495511442423
epoch:  11, loss: 0.00437856325879693
epoch:  12, loss: 0.0035416418686509132
epoch:  13, loss: 0.0029061888344585896
epoch:  14, loss: 0.0023310857359319925
epoch:  15, loss: 0.001947912503965199
epoch:  16, loss: 0.0016874006250873208
epoch:  17, loss: 0.0014796913601458073
epoch:  18, loss: 0.0013282144209370017
epoch:  19, loss: 0.0011659227311611176
epoch:  20, loss: 0.0010186745785176754
epoch:  21, loss: 0.0008995245443657041
epoch:  22, loss: 0.0008015843923203647
epoch:  23, loss: 0.0007184420246630907
epoch:  24, loss: 0.0007011370034888387
epoch:  25, loss: 0.00043

In [16]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.9980894675879775
Test metrics:  R2 = 0.9957860388863368
